In [ ]:
# Libraries
import copy
import numpy as np 
import re

from random import seed
from random import randrange
# seed random number generator
seed(1)

from itertools import combinations

In [ ]:
# We're going to sort out our own dictionary, based on the Collins Scrabble Dictionary text files
WORD_LIST_FILE = 'collins_scrabble_words.txt'
DEFINITION_FILE = 'collins_scrabble_words_defs.txt'

# We want to read through the normal word list and generate an anagram dictionary, 
# to make word lookups faster, given a set of letters ordered alphabetically
ANAGRAMS = {}
f = open(WORD_LIST_FILE, "r")
lines = f.readlines()
for entry in lines[2:]: # The first line is the header
    # Strip the entry into just the word
    word = entry.strip()
    sorted_word = ''.join(sorted(word))
    if sorted_word not in ANAGRAMS.keys():
        ANAGRAMS[sorted_word] = [word]
    else:
        ANAGRAMS[sorted_word].append(word)
f.close

# We want to read through the definition list and pull up all the definitions as strings for our convenience later
DEFINITIONS = {}
f = open(DEFINITION_FILE, "r")
lines = f.readlines()
for entry in lines[2:]: # The first line is the header
    # Split the first word from the definition
    for k in range(len(entry)): 
        if (entry[k].isspace()):
            word = entry[0:k]
            DEFINITIONS[word] = entry[k+1:].strip()            
f.close

# Scrabble constants

# The number of letters in the bag
START_LETTER_DISTRIBUTION = {
    "A":9, "B":2, "C":2, "D":4, "E":12, "F":2, "G":3, "H":2, "I":9, "J":1, "K":1, "L":4, "M":2, 
    "N":6, "O":8, "P":2, "Q":1, "R":6, "S":4, "T":6, "U":4, "V":2, "W":2, "X":1, "Y":2, "Z":1, ".":2}
START_MAX_LETTERS = 0
for key in START_LETTER_DISTRIBUTION:
    START_MAX_LETTERS = START_MAX_LETTERS + START_LETTER_DISTRIBUTION[key]
    
# How much each letter is worth
LETTER_SCORES = {
    ".":0,
    "A":1, "E":1, "I":1, "O":1, "U":1, "L":1, "N":1, "S":1, "T":1, "R":1,
    "D":2, "G":2,
    "B":3, "C":3, "M":3, "P":3,
    "F":4, "H":4, "V":4, "W":4, "Y":4,
    "K":5,
    "J":8, "X":8,
    "Q":10, "Z":10}

# The bonuses for placement on each tile.
# space = no multiplier
# T: triple word, D: double word, t: triple letter, d: double letter
BOARD_SCORES = np.array([
    # 0    1    2    3    4    5    6    7    8    9   10   11   12   13  13
    ['T', ' ', ' ', 'd', ' ', ' ', ' ', 'T', ' ', ' ', ' ', 'd', ' ', ' ', 'T'], # 0
    [' ', 'D', ' ', ' ', ' ', 't', ' ', ' ', ' ', 't', ' ', ' ', ' ', 'D', ' '], # 1
    [' ', ' ', 'D', ' ', ' ', ' ', 'd', ' ', 'd', ' ', ' ', ' ', 'D', ' ', ' '], # 2
    ['d', ' ', ' ', 'D', ' ', ' ', ' ', 'd', ' ', ' ', ' ', 'D', ' ', ' ', 'd'], # 3
    [' ', ' ', ' ', ' ', 'D', ' ', ' ', ' ', ' ', ' ', 'D', ' ', ' ', ' ', ' '], # 4
    [' ', 't', ' ', ' ', ' ', 't', ' ', ' ', ' ', 't', ' ', ' ', ' ', 't', ' '], # 5
    [' ', ' ', 'd', ' ', ' ', ' ', 'd', ' ', 'd', ' ', ' ', ' ', 'd', ' ', ' '], # 6
    ['T', ' ', ' ', 'd', ' ', ' ', ' ', 'D', ' ', ' ', ' ', 'd', ' ', ' ', 'T'], # 7
    [' ', ' ', 'd', ' ', ' ', ' ', 'd', ' ', 'd', ' ', ' ', ' ', 'd', ' ', ' '], # 8
    [' ', 't', ' ', ' ', ' ', 't', ' ', ' ', ' ', 't', ' ', ' ', ' ', 't', ' '], # 9
    [' ', ' ', ' ', ' ', 'D', ' ', ' ', ' ', ' ', ' ', 'D', ' ', ' ', ' ', ' '], # 10
    ['d', ' ', ' ', 'D', ' ', ' ', ' ', 'd', ' ', ' ', ' ', 'D', ' ', ' ', 'd'], # 11
    [' ', ' ', 'D', ' ', ' ', ' ', 'd', ' ', 'd', ' ', ' ', ' ', 'D', ' ', ' '], # 12
    [' ', 'D', ' ', ' ', ' ', 't', ' ', ' ', ' ', 't', ' ', ' ', ' ', 'D', ' '], # 13
    ['T', ' ', ' ', 'd', ' ', ' ', ' ', 'T', ' ', ' ', ' ', 'd', ' ', ' ', 'T']]) # 14

# Starting board state (empty)
m = 15
n = 15
STARTING_BOARD = np.full((m,n), ' ')

In [ ]:
def print_board_state(board):
    print(board)
    return 0

# Generate hands (sets of letters to start with)
def draw_letter(letter_distribution, num_letters_in_bag):
    value = randrange(0, num_letters_in_bag)
    #print('Drawing letter:', value, num_letters_in_bag)
    for key in letter_distribution:
        if value < letter_distribution[key]:
            letter = key
            #print('Found letter', value, letter, letter_distribution[key])
            letter_distribution[letter] = letter_distribution[letter] - 1
            num_letters_in_bag = num_letters_in_bag - 1
            break
        else:
            value = value - letter_distribution[key]
            #print(value, key, letter_distribution[key])
    return letter, letter_distribution, num_letters_in_bag

def get_letters(num_letters, letter_distribution, num_letters_in_bag):
    num_letters = min(num_letters,num_letters_in_bag)
    hand = [''] * num_letters
    for i in range(num_letters):
        if 0 < num_letters_in_bag:
            hand[i], letter_distribution, num_letters_in_bag  = draw_letter(letter_distribution, num_letters_in_bag)
    return hand, letter_distribution, num_letters_in_bag

def remove_tiles(hand, tiles_used):
    # Get rid of each tile used from the hand
    for letter in tiles_used:
        hand.remove(letter)
    return hand

In [ ]:
def put_down_first_word(board, possible_words, scores, ordering_by_score):
    # This function is going to be very simple, and place the word centered (as best as possible)
    # on tile (7,7) (remembering python ordering starts at 0), horizontally (bearing in mind board is symmetric)
    # The best word is naively going to be the one with the highest score, and we'll ignore number of tiles used
    best_word = possible_words[ordering_by_score[0]]
    row = 7
    column = 7 - len(best_word)//2
    for k in range(len(best_word)):
        #print('Placing letter at',row,column,':',best_word[k])
        board[row][column] = best_word[k]
        column = column + 1
    tiles_used = best_word
    return board, tiles_used

# Given a solution, place word on the board and remove from hand
def place_word_from_solution(solution, board, hand):
    # Step through the word, starting from start_coordinate
    coordinate = np.array(copy.copy(solution.start_coordinates))
    step = np.array([1,0]) # Assume vertical
    if solution.is_horizontal:
        step = np.array([0,1]) # horizontal
    for letter in solution.word:
        # Step through the word
        # Is this a letter we are placing down from our hand?
        current_board_value = board[coordinate[0],coordinate[1]]
        if current_board_value == ' ': #If there is not already a letter
            board[coordinate[0],coordinate[1]] = letter
        coordinate = coordinate + step
    #print(hand,solution.tiles_used)
    hand = remove_tiles(hand, solution.tiles_used)
    return board, hand

# Given a board state, and a hand, we want to place a word on the board
def place_word(board_state, hand, first_turn=False, solutions_to_print=1):
    # If first turn is true, we generate possible words from our hand, 
    # figure out the one with the highest score, and place it on the board, 
    # making sure to cover the central tile (8,8).
    # N.B. More complex play might be strategic in this, but for this function, 
    # trying to center the word on the central tile will be enough.
    success = False
    score = 0
    if first_turn:
        possible_words = generate_words(hand)
        if possible_words != []:
            scores, ordering_by_score = calculate_word_scores(possible_words)
            scores = np.array(scores) * 2
            print('There are',len(scores),'possible words:')
            for k in range(0,min(len(scores),solutions_to_print)):#len(scores)):
                print(scores[ordering_by_score[k]], 'pts by placing',possible_words[ordering_by_score[k]], DEFINITIONS[possible_words[ordering_by_score[k]]])
            board_state, tiles_used = put_down_first_word(board_state, possible_words, scores, ordering_by_score)
            hand = remove_tiles(hand, tiles_used)
            success = True
            score = scores[ordering_by_score[k]]
        else:
            print('There are 0 possible words.')
    # If it isn't the first turn, we can only place letters adjacent to existing words on the board
    # so it's not enough just to use the letters in our hand, and all placements must be considered
    else:
        # Find patterns
        patterns = generate_board_patterns(board_state)
        potential_solutions = find_potential_solutions(patterns, board_state, hand)
        
        if potential_solutions != []:
            
            sorted_solutions = sorted(potential_solutions, key=lambda sol: sol.score, reverse=True)
            print('There are',len(sorted_solutions),'possible words:')
            for s in range(0,min(len(sorted_solutions),solutions_to_print)):#len(sorted_solutions)):
                sol = sorted_solutions[s]
                direction = 'vertically'
                if sol.is_horizontal:
                    direction = 'horizontally'
                print(sol.score, 'pts by placing', sol.word, direction, 'from', sol.start_coordinates, '\n\tMeaning:', DEFINITIONS[sol.word])
                for perp in sol.perpendicular_words:
                    print('\t+ ext. ',perp,'meaning:', DEFINITIONS[perp])
                
            # Let's make the move and remove tiles from the hand
            board_state, hand = place_word_from_solution(sorted_solutions[0], board_state, hand)
            success = True
            score = sol.score
        else:
            print('There are 0 possible words.')
    #print(board_state)
    return success, board_state, hand, score

In [ ]:
# Create a type of object to keep track of what kind of word we want to find
class WordPattern:
    def __init__(self, is_horizontal, anchor_coordinate, head_space, anchor_pattern, rest_space, letters):
        self.is_horizontal = is_horizontal # Whether the pattern is horizontal (True) or vertical (False)
        self.anchor_coordinate = anchor_coordinate # The coordinates of the first mandatory letters e.g. [5,14]
        # We break the pattern up into two bits to help us figure out where the word needs to be placed
        #self.head_pattern = head_pattern # e.g. '.{0,7}' - The full pattern would be '.{0,7}[C]..[Y].{0,4}'
        self.head_space = head_space # the max number of letters to place ahead of anchor
        self.anchor_pattern = anchor_pattern # e.g. '[C]..[Y]'
        #self.rest_pattern = rest_pattern # e.g. '.{0,4}'
        self.rest_space = rest_space # the max number of letters to place after anchor
        self.letters = letters # e.g. ['C','Y']
    
class Solution:
    def __init__(self, start_coordinates, is_horizontal, word, pattern):
        self.start_coordinates = start_coordinates # e.g. [5,7]
        #self.anchor_coordinate = anchor_coordinate # The coordinates of the mandatory letters e.g. [5,14]
        self.is_horizontal = is_horizontal # Whether the pattern is horizontal (True) or vertical (False)
        self.word = word # The solution itself e.g. 'CATAPULT'
        self.tiles_used = [] # e.g. ['A','P','U', 'L', 'T']
        self.score = 0 # e.g. 50
        self.pattern = pattern # This is for debugging only.
        self.perpendicular_words = []

def get_sequence(row_or_column):
    sequence = []
    gap = 0
    letters = ''
    finding_spaces = True
    for c in row_or_column:
        if c == ' ':
            if not finding_spaces:
                sequence.append(letters)
                finding_spaces = not finding_spaces
                letters = ''
            gap += 1
        else:
            if finding_spaces:
                sequence.append(gap)
                finding_spaces = not finding_spaces
                gap = 0
            letters += c
    if finding_spaces:
        sequence.append(gap)
    else:
        sequence.append(letters)
    #print(sequence)
    return sequence
        
def generate_patterns_from_sequence(is_horizontal, row_or_column_number, sequence):
    # [ g0 l1 g2]
    # We know there are gaps at either end. They are even indices 0 to len()-1
    #print('Sequence:',sequence)
    patterns = []
    if 1 < len(sequence):
        for s in range(1,len(sequence)-1,2):
            for e in range(s,len(sequence)-1,2):
                head_space = sequence[s-1]-1 if s != 1 else sequence[s-1]
                anchor_coordinate = [-1,-1]
                anchor_coordinate[0 if is_horizontal else 1] = row_or_column_number # coordinate given as input
                anchor_coordinate[1 if is_horizontal else 0] = sum([a if isinstance(a, int) else len(a) for a in sequence[:s]]) # from first gap + 1
                anchor_pattern = ''.join(['.' * a if isinstance(a, int) else str(a) for a in sequence[s:e+1]])
                rest_space = sequence[e+1]-1 if e != len(sequence)-2 else sequence[e+1]
                letters = list(''.join([l if isinstance(l,str) else '' for l in sequence[s:e+1]]))
                patterns.append(WordPattern(is_horizontal, anchor_coordinate, head_space, anchor_pattern, rest_space, letters))
                #print('Pattern:',is_horizontal, anchor_coordinate, head_space, anchor_pattern, rest_space, letters)
    return patterns
    
def generate_board_patterns(board):
    # For each row, and then each column, create sequences [gap, letters, gap]
    patterns = []
    for row in range(0,board.shape[0]):
        patterns.extend(generate_patterns_from_sequence(True,row,get_sequence(board[row,:])))
    for column in range(0,board.shape[1]):
        patterns.extend(generate_patterns_from_sequence(False,column,get_sequence(board[:,column])))
    return patterns
    
def get_perpendicular_word(board, letter_being_placed, coordinate, is_horizontal):
    step = np.array([0,1]) # Since word is assumed vertical, we assume horizontal perpendicular words
    if is_horizontal:
        step = np.array([1,0]) # horizontal word -> vertical perpendicular words
    perpendicular_word = letter_being_placed # the starting letter
    
    #print('direction',step,'starting with',perpendicular_word,'at',coordinate)
    
    # Check upstream and add letters to the front until we can't.
    upstream = coordinate - step
    done = False
    while 0 <= upstream[0] and 0 <= upstream[1] and not done:
        upstream_letter = board[upstream[0],upstream[1]]
        #print('upstream',upstream_letter,'at',upstream)
        if upstream_letter == ' ':
            done = True
        else:
            perpendicular_word = upstream_letter + perpendicular_word
            upstream = upstream - step
            #print('moved to',upstream)
        
    # Check downstream and add letters to the back until we can't.
    downstream = coordinate + step
    done = False
    while downstream[0] < 15 and downstream[1] < 15 and not done:
        downstream_letter = board[downstream[0],downstream[1]]
        #print('downstream',downstream_letter,'at',downstream)
        if downstream_letter == ' ':
            done = True
        else:
            perpendicular_word = perpendicular_word + downstream_letter
            downstream = downstream + step
            #print('moved to',downstream)
    
    if letter_being_placed == perpendicular_word: #i.e. no word found
        perpendicular_word = ''
        
    return perpendicular_word
        
# Given letters in a hand, find all possible words
def generate_words(hand, mandatory_letters = [], max_choices = float('inf')):
    
    # We need to consider using subsets of the hand
    letter_subsets = []
    for r in range(1,min(len(hand)+1,max_choices+1)): 
        letter_subsets.extend(list(combinations(sorted(hand), r)))
    
    # Remove duplicates
    letter_subsets = list(dict.fromkeys(letter_subsets))
    letter_subsets = list(dict.fromkeys(letter_subsets)) 
    #print('letter_subsets:\n',mylist)
    
    possible_words = []
    for letters in letter_subsets:
        all_letters = []
        all_letters.extend(letters)
        all_letters.extend(mandatory_letters)
        key = ''.join(sorted(all_letters))
        
        if key in ANAGRAMS:
            possible_words.extend(ANAGRAMS[key])
    return possible_words

# Given words, calculate scores, and sort the list by score
# Only takes the letter values into account, no board values, or any existing words we've added to
def calculate_word_scores(words):
    scores = []
    for word in words:
        score = 0
        for letter in word:
            score += LETTER_SCORES[letter]
        scores.append(score)
    #print("scores:",scores)
    ordering_by_score = sorted(range(len(scores)), key=lambda k: scores[k], reverse=True)
    #print("ordering_by_score:",ordering_by_score)
    return scores, ordering_by_score

# Given a solution, which is a word, and a position on the board, what is the score?
# This should depend on:
# 1. the letters
# 2. whether tiles on the board offer multipliers
# 3. whether we get a 50pt bonus for using 7 letters
# 4. extra points for extending words already on the board
# TODO: We need to add scores for horizontal/vertical offshoots?
def calculate_solution_scores(solutions, board):
    validated_solutions = []
    for sol in solutions:
        #print('Calculating score for:',sol.word)
        score = 0
        perpendicular_word_scores = 0;
        word_multiplier = 1
        sol.tiles_used = []
        # Step through the word, starting from start_coordinate
        solution_valid = True
        coordinate = np.array(copy.copy(sol.start_coordinates))
        step = np.array([1,0]) # Assume vertical
        if sol.is_horizontal:
            step = np.array([0,1]) # horizontal
        for letter in sol.word:
            # Step through the word
            # Is this a letter we are placing down from our hand?
            current_board_value = board[coordinate[0],coordinate[1]]
            #print('\tStepping through solution score:',letter,coordinate,current_board_value,'multiplier:',BOARD_SCORES[coordinate[0],coordinate[1]])
            if current_board_value != ' ': #If there is already a letter (no multipliers apply)
                #print('These should be the same:',current_board_value,letter)
                score += LETTER_SCORES[letter]
            else: # Then we are placing a letter from our own hand down. 
                  # We would use the tile, and get any score multiplier from the tile
                if BOARD_SCORES[coordinate[0],coordinate[1]] == 'T':
                    #print('\tTriple word')
                    word_multiplier = word_multiplier * 3
                    score += LETTER_SCORES[letter]
                elif BOARD_SCORES[coordinate[0],coordinate[1]] == 'D':
                    #print('\tDouble word')
                    word_multiplier = word_multiplier * 2
                    score += LETTER_SCORES[letter]
                elif BOARD_SCORES[coordinate[0],coordinate[1]] == 't':
                    #print('\tTriple letter')
                    score += LETTER_SCORES[letter] * 3
                elif BOARD_SCORES[coordinate[0],coordinate[1]] == 'd':
                    #print('\tDouble letter')
                    score += LETTER_SCORES[letter] * 2
                else: # no multiplier
                    score += LETTER_SCORES[letter]
                #print('\tTile placed:',letter)
                sol.tiles_used.append(letter)
                
                # Since we are placing a letter, we need to check for perpendicular words
                # Check the tiles on either side (or above/below) for contiguous words.
                # Are the words we've extended in the dictionary?
                #print('Checking for perpendicular words',coordinate)
                perpendicular_word = get_perpendicular_word(board,letter,coordinate,sol.is_horizontal)
                #print('perp word:',perpendicular_word)
                if perpendicular_word in DEFINITIONS:
                    #print('Found:',perpendicular_word)
                    pscores, p_ordering_by_score = calculate_word_scores([perpendicular_word])
                    #print(pscores)
                    pscore = pscores[0]
                    #print(pscore)
                    if BOARD_SCORES[coordinate[0],coordinate[1]] == 'T':
                        #print('\tTriple word')
                        pscore = pscore * 3
                    elif BOARD_SCORES[coordinate[0],coordinate[1]] == 'D':
                        #print('\tDouble word')
                        pscore = pscore * 2
                    elif BOARD_SCORES[coordinate[0],coordinate[1]] == 't':
                        #print('\tTriple letter')
                        pscore += LETTER_SCORES[letter] * 2
                    elif BOARD_SCORES[coordinate[0],coordinate[1]] == 'd':
                        #print('\tDouble letter')
                        pscore += LETTER_SCORES[letter] * 1
                    perpendicular_word_scores += pscore
                 
                    sol.perpendicular_words.append(perpendicular_word)
                else:
                    #print('What happened to this word?',perpendicular_word)
                    if perpendicular_word != '':
                        solution_valid = False # Placing this letter would create a nonsense word
                
            coordinate = coordinate + step
                
        # Apply the cumulative word multiplier
        #print('\tWord multiplier:',word_multiplier)
        score = score * word_multiplier
        
        # Add all the perpendicular word scores in
        score += perpendicular_word_scores
        
        # Did we use seven (7) letters? If so, add 50 points.
        if len(sol.tiles_used) == 7:
            score += 50
        
        sol.score = score
        
        if solution_valid:
            validated_solutions.append(sol)
        
        #print('Score:',sol.score,'Tiles used:',sol.tiles_used)
    return validated_solutions # Since we are only adjusting the score stored in the object

def find_potential_solutions(patterns, board, hand):
    potential_solutions = []
    for pat in patterns:
        pat.possible_words = []
        full_pattern = '.{0,',pat.head_space,'}' + pat.anchor_pattern + '.{0,',pat.rest_space,'}'
        #print('TRYING TO MATCH PATTERN:',full_pattern, 'with:', hand, pat.letters, 'at', pat.anchor_coordinate)
        possible_words = generate_words(hand, pat.letters)
        if possible_words != []:
            for word in possible_words:
                #print('Checking new word:',word)
                # How many times do we find the anchor of the pattern?
                m = re.search(pat.anchor_pattern, word)
                word_copy = word
                offset = 0
                while m:
                    #print('Searching',word,'we found the anchor at',m.start()+offset)
                    # Take out the anchor from the word, and match the
                    # header and tail segments to the relevant patterns
                    #print(word,'m.start',m.start(),'offset',offset,'len(pat.anchor_pattern)',len(pat.anchor_pattern))
                    word_header = word[:m.start()+offset]
                    word_anchor = word[m.start()+offset:m.start()+offset+len(pat.anchor_pattern)]
                    word_tail = word[m.start()+offset+len(pat.anchor_pattern):]
                    #print('Full word:',word,'offset:',offset,'truncated word:',word_copy,'h/a/t:',word_header,'/',word_anchor,'(',pat.anchor_pattern,')/', word_tail)
                    if len(word_header) <= pat.head_space and len(word_tail) <= pat.rest_space: # both match
                        # Add the word as a solution, with the correct placement provided
                        #print('\tValid placement:')
                        start_coordinates = copy.copy(pat.anchor_coordinate)
                        if pat.is_horizontal:
                            start_coordinates[1] -= len(word_header)
                            #print('\tAdjusting horizontally by', len(word_header), start_coordinates)
                        else:
                            start_coordinates[0] -= len(word_header)
                            #print('\tAdjusting vertically by', len(word_header), start_coordinates)
                        new_solution = Solution(start_coordinates, pat.is_horizontal, word, pat)
                        #print(new_solution.score, 'pts by placing', new_solution.word, pat.is_horizontal, 'from', new_solution.start_coordinates)
                        #new_solution.tiles_used = 
                        potential_solutions.append(new_solution)
                    word_copy = word_copy[m.start()+1:]
                    #print('\tTruncating to try and find:',pat.anchor_pattern,'in',word_copy)
                    offset += m.start() + 1
                    m = re.search(pat.anchor_pattern, word_copy)
        #print('There are',len(pat.possible_words),'possible words.')

    # We need to delete any solutions that are invalidated by neighbouring tiles
    # The space before and after the word will have been dealt with by the pattern generation for that row/column
    # We need to check either the rows above and below, or the columns to either side    
    validated_solutions = calculate_solution_scores(potential_solutions, board)
    
    return validated_solutions

In [ ]:
# Testing generating patterns
pat = generate_patterns_from_sequence(True, 1, [5,'A',5])
for p in pat:
    print('1:',p.is_horizontal,p.anchor_coordinate,p.head_space,p.anchor_pattern,p.rest_space,p.letters)

pat = generate_patterns_from_sequence(True, 1, [15])
for p in pat:
    print('2:',p.is_horizontal,p.anchor_coordinate,p.head_space,p.anchor_pattern,p.rest_space,p.letters)

pat = generate_patterns_from_sequence(True, 1, [1,'A',3,'BC',5,'D',0])
# 1:'A'
for p in pat:
    print('3:',p.is_horizontal,p.anchor_coordinate,p.head_space,p.anchor_pattern,p.rest_space,p.letters)
    
pat = generate_patterns_from_sequence(False, 1, [0,'A',3,'BC',5,'D',6])
for p in pat:
    print('4:',p.is_horizontal,p.anchor_coordinate,p.head_space,p.anchor_pattern,p.rest_space,p.letters)

In [ ]:
# Start a board state and hand, and list all the possible moves, taking the best one
print('Creating board...')
_board = np.array([
    # 0    1    2    3    4    5    6    7    8    9   10   11   12   13  13
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 0
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 1
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 2
    [' ', ' ', ' ', 'R', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 3
    [' ', ' ', ' ', 'A', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 4
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 5
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 6
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 7
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 8
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 9
    [' ', ' ', ' ', 'R', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 10
    [' ', ' ', ' ', 'A', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 11
    [' ', ' ', ' ', 'S', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 12
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], # 13
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]) # 14

print('Putting tiles in the bag...')
_letter_bag = copy.copy(START_LETTER_DISTRIBUTION)
_num_letters_in_bag = copy.copy(START_MAX_LETTERS)
_num_letters = 7 # How many letters we have at each round
_hand = [];
_success = False

# Let's get the first hand
_hand = ['A','B','C','A','D','A','B']

# Calculate the next move
_success, _board, _hand, tmp_score= place_word(_board, _hand, False, float('inf'))
print_board_state(_board)
print('Hand after move:', _hand)

In [ ]:
# Start a game with one player
seed(1) # We want to have a predictable problem
print('Creating blank board...')
_board = copy.copy(STARTING_BOARD)

print('Putting tiles in the bag...')
_letter_bag = copy.copy(START_LETTER_DISTRIBUTION)
_num_letters_in_bag = copy.copy(START_MAX_LETTERS)
_num_letters = 7 # How many letters we have at each round
_hand = [];
_success = False

# Let's get the first hand
print('Picking',_num_letters-len(_hand),'tiles out of the bag...')
_new_tiles, _letter_bag, _num_letters_in_bag = get_letters(_num_letters-len(_hand), _letter_bag, _num_letters_in_bag)
_hand.extend(_new_tiles)
print('Hand:', _hand, 'Letters left in bag:',_num_letters_in_bag)

# Let's take the first move
_success, _board, _hand, tmp_score = place_word(_board, _hand, True)
print_board_state(_board)
print('Hand after move:', _hand)

# Replenish hand
rounds = 0
while _success:
    print('Picking',_num_letters-len(_hand),'tiles out of the bag...')
    _new_tiles, _letter_bag, _num_letters_in_bag = get_letters(_num_letters-len(_hand), _letter_bag, _num_letters_in_bag)
    _hand.extend(_new_tiles)
    print('Hand:', _hand, 'Letters left in bag:',_num_letters_in_bag)

    # Calculate the next move
    _success, _board, _hand, tmp_score= place_word(_board, _hand, False)
    print_board_state(_board)
    print('Hand after move:', _hand)
    
    rounds += 1


In [ ]:
# Start a game with four players
print('Creating blank board...')
_board = copy.copy(STARTING_BOARD)

print('Putting tiles in the bag...')
_letter_bag = copy.copy(START_LETTER_DISTRIBUTION)
_num_letters_in_bag = copy.copy(START_MAX_LETTERS)
_num_letters = 7 # How many letters we have at each round
_game_over = False

# Set up the four players
class Player:
    def __init__(self,name):
        self.name = name
        self.hand = []
        self.score = 0
        self.passes_in_a_row = 0

_four_players = [Player('Anna'), Player('Bob'), Player('Carl'), Player('Dixie')]
        
# Let's get the four hand
for player in _four_players:
    print(player.name,'is picking',_num_letters-len(player.hand),'tiles out of the bag...')
    _new_tiles, _letter_bag, _num_letters_in_bag = get_letters(_num_letters-len(player.hand), _letter_bag, _num_letters_in_bag)
    player.hand.extend(_new_tiles)
    print('\tHand:', player.hand, 'Letters left in bag:',_num_letters_in_bag)

# The first player makes their first move
# We make this player 4 so that the round order of (1-4) after that is correct
player = _four_players[3]

print(player.name,'\'s move...')
_success, _board, player.hand, _score = place_word(_board, player.hand, True)
player.score += _score
print_board_state(_board)
print('\tHand after move:', player.hand)

print('\t',player.name,'is picking',_num_letters-len(player.hand),'tiles out of the bag...')
_new_tiles, _letter_bag, _num_letters_in_bag = get_letters(_num_letters-len(player.hand), _letter_bag, _num_letters_in_bag)
player.hand.extend(_new_tiles)
print('\tHand:', player.hand, 'Letters left in bag:',_num_letters_in_bag)
    
# Replenish hand
while not _game_over:
    
    for player in _four_players:
        
        if not _game_over:
            # Calculate the next move
            print(player.name,'\'s move...')
            _success, _board, player.hand, _score = place_word(_board, player.hand, False)
            player.score += _score
            print_board_state(_board)
            print('\tHand after move:', player.hand)
            if _success:
                player.passes_in_a_row = 0
            else:
                player.passes_in_a_row += 1
                # END GAME CONDITION: Check if every player has passed twice in a row
                everyone_passed_twice = True
                for player in _four_players:
                    if player.passes_in_a_row < 2:
                        everyone_passed_twice = False
                if everyone_passed_twice:
                    _game_over = True
            
            print('\t',player.name,'is picking',_num_letters-len(player.hand),'tiles out of the bag...')
            _new_tiles, _letter_bag, _num_letters_in_bag = get_letters(_num_letters-len(player.hand), _letter_bag, _num_letters_in_bag)
            player.hand.extend(_new_tiles)
            print('\tHand:', player.hand, 'Letters left in bag:',_num_letters_in_bag)
            
            # END GAME CONDITION: Did someone empty their hand and there are no more tiles to pick?
            if player.hand == [] and _num_letters_in_bag == 0:
                _game_over = True
                
print('Game over.')

_four_players = sorted(_four_players, key=lambda p: p.score, reverse=True)
for player in _four_players:
    print(player.name,'scored',player.score,'points. (Remaining tiles:',player.hand,')')